In [1]:
#数据读取 为了方便大家更好的理解数据，我们重命名为中文
import numpy  as np
import pandas as pd
import os
# pandasql 可以把SQL代码写到这里，有些复杂的逻辑方便SQL写，并且日常应用也好参考,没有的安装下 pip install pandasql
from   pandasql import sqldf


data      = pd.read_csv("账户交易信息.csv", encoding="gbk")
train     = pd.read_csv("训练集标签.csv", encoding="gbk")
user_info = pd.read_csv("账户静态信息.csv", encoding="utf-8")
test      = pd.read_csv("test_dataset.csv", encoding="utf-8")
data = data.rename({
                    'jylsxh': '交易流水序号',
                    'zhdh': '账户代号',
                    'dfzh': '对方账号',
                    'jdbz': '借贷标志',
                    'jyje': '交易金额',
                    'jyye': '交易余额',
                    'dfhh': '对方行号',
                    'jyrq': '交易日期',
                    'jysj': '交易时间',
                    'jyqd': '交易渠道',
                    'zydh': '摘要代号',
                    'jdbj': '借贷标记',
                    'zhye': '账户余额',
                    'dfmccd': '对方名称长度'}, axis=1
                  )

train    = train.rename({'zhdh': '账户代号', 
                         'black_flag': '标签'}, axis=1
                       )

user_info =  user_info.rename({'zhdh': '账户代号',
                               'khrq':'开户日期', 
                               'xb': '性别', 
                             'khjgdh':'开户机构代号'}, axis=1
                              )

test    =  test.rename({'zhdh': '账户代号'},axis=1)

# #对数据进行聚合
# df = data.groupby(['账户代号','对方账号']).agg({'交易流水序号': pd.Series.nunique}).reset_index()
# print('聚合数据总数:\n',df.shape)

# # 剔除自己指向自己的节点
# df = df[df['账户代号']!=df['对方账号']]
# print('剔除后的数据:\n',df.shape)

In [2]:
# 异构网络构建
df1 = data[['账户代号','对方账号']]
df1.columns = ['账户代号','对方账号']


df2 = data[['账户代号','摘要代号']]
df2.columns = ['账户代号','对方账号']


# 关系合并
df = pd.concat([df1,df2])


# 对数据进行聚合
df = df.groupby(['账户代号','对方账号']).agg({'对方账号': ['count']}).reset_index()
df.columns=['账户代号','对方账号','CNT']

# 剔除自己指向自己的节点
print('剔除前的数据:\n',df.shape)

df = df[df['账户代号']!=df['对方账号']]
print('剔除后的数据:\n',df.shape)


剔除前的数据:
 (239495, 3)
剔除后的数据:
 (238808, 3)


In [3]:
# 关系数据转换成networkx的图格式
import networkx as nx
import matplotlib.pyplot as plt

# da = join_table[join_table['sim']>=0]
# print(da.shape)
da = df[['账户代号','对方账号']].values
G  = nx.Graph()
for num in range(len(da)):
    G.add_edge(str(da[num,0]),str(da[num,1]))
    

In [5]:
from node2vec import Node2Vec
from datetime import datetime

#加个运行时间的监控，这里是起点
start = datetime.now()

#参数设置
node2vec = Node2Vec(G,
                    dimensions=128, 
                    walk_length=10, 
                    p=0.8,
                    q=1.2,
                    num_walks=10, 
                    workers=4
                    ) 

#embedding 训练
model = node2vec.fit(window    = 8, 
                     min_count = 1, 
                     batch_words = 4
                    )

#加个运行时间的监控，这里是终点
end = datetime.now()
# 打印运行时间差
print(end - start)

Computing transition probabilities:   0%|          | 0/137592 [00:00<?, ?it/s]